# 1. 데이터 준비하기

### 패키지 설치하기

In [1]:
!pip install koreanize-matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 81.6 MB/s eta 0:00:00


### 라이브러리 임포트

In [5]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

import folium

import warnings
warnings.filterwarnings('ignore')

In [4]:
pip install requests pandas tqdm


### 데이터 불러오기

```
# 코드로 형식 지정됨
```



In [20]:
#@title 설치 (requests, pandas, tenacity 등)
!pip -q install requests pandas python-dateutil tenacity tqdm pyyaml

import os, json, time, math
from typing import Any, Dict, Optional, Iterable, List, Set
from datetime import datetime, timedelta
import pandas as pd
import requests
from dateutil.parser import isoparse
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from tqdm import tqdm

DATA_DIR = "/content/data"  # 저장 경로
os.makedirs(DATA_DIR, exist_ok=True)

RAW_DAILY_CSV = f"{DATA_DIR}/boxoffice_daily.csv"
META_CSV = f"{DATA_DIR}/movie_meta_kobis.csv"
GENRES_CSV = f"{DATA_DIR}/movie_genre.csv"
DIRECTORS_CSV = f"{DATA_DIR}/movie_director.csv"
ACTORS_CSV = f"{DATA_DIR}/movie_actor.csv"
MONTHLY_CSV = f"{DATA_DIR}/boxoffice_monthly.csv"

print("환경 준비 완료 ✅")


환경 준비 완료 ✅


In [22]:
#@title 설정(반드시 API 키 입력)
KOBIS_API_KEY = "92c4539ff855e8fa10b667835f43fc48"  #@param {type:"string"}
PER_REQUEST_SLEEP = 0.35           #@param {type:"number"}
MAX_RETRIES = 3                    #@param {type:"integer"}

# 수집 기간 (요청 범위 고정)
DATE_START = "2015-01-01"          #@param {type:"string"}
DATE_END   = "2025-09-30"          #@param {type:"string"}

# 스모크 테스트(짧게 검증하고 전체 실행): True면 7일만 먼저 수집
SMOKE_TEST_FIRST = True            #@param {type:"boolean"}

assert KOBIS_API_KEY and KOBIS_API_KEY != "여기에_키_입력", "❌ KOBIS_API_KEY를 입력하세요."
print("설정 확인 완료 ✅")


설정 확인 완료 ✅


In [23]:
#@title KOBIS 클라이언트 정의
class KobisError(Exception):
    pass

class KobisClient:
    def __init__(self, api_key: str, base_url: str = "https://www.kobis.or.kr/kobisopenapi/webservice/rest",
                 per_request_sleep: float = 0.35, max_retries: int = 3):
        if not api_key:
            raise KobisError("KOBIS_API_KEY가 비어있습니다.")
        self.api_key = api_key
        self.base_url = base_url.rstrip("/")
        self.per_request_sleep = float(per_request_sleep)
        self.max_retries = int(max_retries)
        self.session = requests.Session()
        self.session.headers.update({"User-Agent": "kobis-colab-pipeline/1.0"})

    def _url(self, path: str) -> str:
        return f"{self.base_url}/{path.lstrip('/')}"

    @retry(
        reraise=True,
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=0.5, min=0.5, max=4),
        retry=retry_if_exception_type((requests.RequestException, KobisError)),
    )
    def get_json(self, path: str, params: Dict[str, Any]) -> Dict[str, Any]:
        # 동적으로 재시도 횟수를 반영
        if self.max_retries != 3:
            self.get_json.retry.stop = stop_after_attempt(self.max_retries)  # type: ignore
        p = dict(params or {})
        p.setdefault("key", self.api_key)
        url = self._url(path)
        resp = self.session.get(url, params=p, timeout=20)
        if resp.status_code >= 500:
            raise requests.RequestException(f"Server error: {resp.status_code}")
        if resp.status_code != 200:
            raise KobisError(f"HTTP {resp.status_code}: {resp.text[:200]}")
        data = resp.json()
        # 레이트리밋 슬립
        time.sleep(self.per_request_sleep)
        return data

client = KobisClient(
    api_key=KOBIS_API_KEY,
    per_request_sleep=PER_REQUEST_SLEEP,
    max_retries=MAX_RETRIES
)
print("KOBIS 클라이언트 준비 완료 ✅")


KOBIS 클라이언트 준비 완료 ✅


In [24]:
#@title 유틸 함수들
def date_range_inclusive(start: str, end: str) -> Iterable[str]:
    s = isoparse(start).date()
    e = isoparse(end).date()
    d = s
    while d <= e:
        yield d.strftime("%Y%m%d")
        d += timedelta(days=1)

def ensure_parent_dir(path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)

def now_utc_iso() -> str:
    return datetime.utcnow().isoformat(timespec="seconds") + "Z"

def upsert_csv(df_new: pd.DataFrame, out_csv: str, key_cols: List[str], sort_cols: List[str]):
    ensure_parent_dir(out_csv)
    if os.path.exists(out_csv):
        old = pd.read_csv(out_csv, dtype=str)
        all_df = pd.concat([old, df_new], ignore_index=True)
        all_df = all_df.sort_values(sort_cols).drop_duplicates(subset=key_cols, keep="last")
    else:
        all_df = df_new.sort_values(sort_cols)
    all_df.to_csv(out_csv, index=False, encoding="utf-8")


In [25]:
#@title 일별 수집 함수 + 실행(스모크→전체 순차)
DAILY_PATH = "boxoffice/searchDailyBoxOfficeList.json"

def parse_daily_payload(targetDt: str, payload: Dict) -> List[Dict]:
    box = payload.get("boxOfficeResult", {}).get("dailyBoxOfficeList", [])
    rows = []
    for item in box:
        rows.append({
            "date": f"{targetDt[:4]}-{targetDt[4:6]}-{targetDt[6:]}",
            "movieCd": item.get("movieCd"),
            "movieNm": item.get("movieNm"),
            "rank": item.get("rank"),
            "rankInten": item.get("rankInten"),
            "rankOldAndNew": item.get("rankOldAndNew"),
            "salesAmt": item.get("salesAmt"),
            "salesShare": item.get("salesShare"),
            "salesAcc": item.get("salesAcc"),
            "audiCnt": item.get("audiCnt"),
            "audiAcc": item.get("audiAcc"),
            "scrnCnt": item.get("scrnCnt"),
            "showCnt": item.get("showCnt"),
            "collected_at": now_utc_iso(),
        })
    return rows

def fetch_daily_range_to_csv(client: KobisClient, start: str, end: str, out_csv: str):
    all_rows: List[Dict] = []
    days = list(date_range_inclusive(start, end))
    for yyyymmdd in tqdm(days, desc=f"Daily {start}~{end}"):
        payload = client.get_json(DAILY_PATH, {"targetDt": yyyymmdd})
        all_rows.extend(parse_daily_payload(yyyymmdd, payload))
    if not all_rows:
        print("빈 결과. 저장 생략.")
        return
    df_new = pd.DataFrame(all_rows)

    # dtype 정제
    int_cols = ["rank","rankInten","salesAmt","salesAcc","audiCnt","audiAcc","scrnCnt","showCnt"]
    float_cols = ["salesShare"]
    for c in int_cols:
        df_new[c] = pd.to_numeric(df_new[c], errors="coerce").astype("Int64")
    for c in float_cols:
        df_new[c] = pd.to_numeric(df_new[c], errors="coerce")

    upsert_csv(df_new, out_csv, key_cols=["date","movieCd"], sort_cols=["date","movieCd","collected_at"])

# --- 실행(스모크 테스트 → 전체) ---
if SMOKE_TEST_FIRST:
    # 7일 샘플 (시작일부터 6일)
    s = isoparse(DATE_START).date()
    e = (s + timedelta(days=6)).isoformat()
    print(f"스모크 테스트 수집: {DATE_START} ~ {e}")
    fetch_daily_range_to_csv(client, DATE_START, e, RAW_DAILY_CSV)
    # 간단히 확인
    display(pd.read_csv(RAW_DAILY_CSV).head(20))

print("전체 수집 진행...")
fetch_daily_range_to_csv(client, DATE_START, DATE_END, RAW_DAILY_CSV)
print(f"일별 수집 완료 ✅ → {RAW_DAILY_CSV}")


스모크 테스트 수집: 2015-01-01 ~ 2015-01-07


Daily 2015-01-01~2015-01-07: 100%|██████████| 7/7 [00:07<00:00,  1.00s/it]


,date,movieCd,movieNm,rank,rankInten,rankOldAndNew,salesAmt,salesShare,salesAcc,audiCnt,audiAcc,scrnCnt,showCnt,collected_at
0,2015-01-01,20130574,개를 훔치는 완벽한 방법,7,2,OLD,299346400,2.0,608002300,37741,80633,196,650,2025-10-30T02:54:06Z
1,2015-01-01,20137048,국제시장,1,0,OLD,6129982500,40.5,47219550096,751754,6097431,941,4650,2025-10-30T02:54:06Z
2,2015-01-01,20140226,호빗: 다섯 군대 전투,6,0,OLD,609678800,4.0,22038162144,69988,2579875,346,825,2025-10-30T02:54:06Z
3,2015-01-01,20141111,"님아, 그 강을 건너지 마오",4,1,OLD,1479015600,9.8,31324845079,178941,4025319,473,1979,2025-10-30T02:54:06Z
4,2015-01-01,20143344,눈의 여왕 2: 트롤의 마법거울,8,0,OLD,233190700,1.5,3856612400,30978,529298,246,415,2025-10-30T02:54:06Z
5,2015-01-01,20143642,테이큰 3,2,1,OLD,2644551100,17.5,3658460100,321653,467280,614,2947,2025-10-30T02:54:06Z
6,2015-01-01,20147176,상의원,9,-2,OLD,230484800,1.5,5524209364,28439,705071,258,491,2025-10-30T02:54:06Z
7,2015-01-01,20149120,인터스텔라,10,0,OLD,192143100,1.3,80863567400,20636,10125883,97,175,2025-10-30T02:54:06Z
8,2015-01-01,20149265,기술자들,5,-1,OLD,1299883700,8.6,14952363848,160371,1938454,464,1886,2025-10-30T02:54:06Z
9,2015-01-01,20149859,마다가스카의 펭귄,3,-1,OLD,1687516200,11.2,2736013000,212779,361669,594,1846,2025-10-30T02:54:06Z


전체 수집 진행...


Daily 2015-01-01~2025-09-30: 100%|██████████| 3926/3926 [59:04<00:00,  1.11it/s]


일별 수집 완료 ✅ → /content/data/boxoffice_daily.csv


In [26]:
#@title 메타 수집/정규화
META_PATH = "movie/searchMovieInfo.json"

def movie_info_row(movieCd: str, payload: Dict) -> Dict:
    info = payload.get("movieInfoResult", {}).get("movieInfo", {})
    def as_json(x):
        return json.dumps(x, ensure_ascii=False, separators=(",",":"))
    return {
        "movieCd": movieCd,
        "movieNm": info.get("movieNm"),
        "movieNmEn": info.get("movieNmEn"),
        "openDt": info.get("openDt"),
        "prdtYear": info.get("prdtYear"),
        "showTm": info.get("showTm"),
        "typeNm": info.get("typeNm"),
        "audits": as_json(info.get("audits", [])),
        "nations": as_json(info.get("nations", [])),
        "genres": as_json(info.get("genres", [])),
        "directors": as_json(info.get("directors", [])),
        "actors": as_json(info.get("actors", [])),
        "companys": as_json(info.get("companys", [])),
        "collected_at": now_utc_iso(),
    }

def fetch_meta_for_missing_to_csv(client: KobisClient, daily_csv: str, out_csv: str):
    assert os.path.exists(daily_csv), f"daily csv 없음: {daily_csv}"
    daily = pd.read_csv(daily_csv, dtype=str, usecols=["movieCd"])
    codes = sorted(set(daily["movieCd"].dropna().tolist()))
    existing_codes: Set[str] = set()
    if os.path.exists(out_csv):
        existing = pd.read_csv(out_csv, dtype=str, usecols=["movieCd"])
        existing_codes = set(existing["movieCd"].dropna().tolist())
    target = [c for c in codes if c not in existing_codes]
    if not target:
        print("추가 메타 대상 없음.")
        return
    rows = []
    for movieCd in tqdm(target, desc="Meta fetch"):
        payload = client.get_json(META_PATH, {"movieCd": movieCd})
        rows.append(movie_info_row(movieCd, payload))
    df_new = pd.DataFrame(rows)
    upsert_csv(df_new, out_csv, key_cols=["movieCd"], sort_cols=["movieCd","collected_at"])

def normalize_meta_to_csvs(meta_csv: str, out_genres: str, out_directors: str, out_actors: str):
    import json as _json
    meta = pd.read_csv(meta_csv, dtype=str)
    genres_rows, directors_rows, actors_rows = [], [], []
    for _, row in meta.iterrows():
        movieCd = row["movieCd"]
        for g in _json.loads(row.get("genres","[]")):
            name = g.get("genreNm")
            if name: genres_rows.append({"movieCd": movieCd, "genreNm": name})
        for d in _json.loads(row.get("directors","[]")):
            name = d.get("peopleNm")
            if name: directors_rows.append({"movieCd": movieCd, "peopleNm": name})
        for a in _json.loads(row.get("actors","[]")):
            name = a.get("peopleNm")
            cast = a.get("cast")
            if name: actors_rows.append({"movieCd": movieCd, "peopleNm": name, "cast": cast})
    if genres_rows:
        pd.DataFrame(genres_rows).drop_duplicates().to_csv(GENRES_CSV, index=False, encoding="utf-8")
    if directors_rows:
        pd.DataFrame(directors_rows).drop_duplicates().to_csv(DIRECTORS_CSV, index=False, encoding="utf-8")
    if actors_rows:
        pd.DataFrame(actors_rows).drop_duplicates().to_csv(ACTORS_CSV, index=False, encoding="utf-8")

# --- 실행 ---
fetch_meta_for_missing_to_csv(client, RAW_DAILY_CSV, META_CSV)
normalize_meta_to_csvs(META_CSV, GENRES_CSV, DIRECTORS_CSV, ACTORS_CSV)

print("메타 보강/정규화 완료 ✅")
display(pd.read_csv(META_CSV).head(10))


Meta fetch: 100%|██████████| 2176/2176 [22:45<00:00,  1.59it/s]

메타 보강/정규화 완료 ✅


,movieCd,movieNm,movieNmEn,openDt,prdtYear,showTm,typeNm,audits,nations,genres,directors,actors,companys,collected_at
0,19528001,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],[],[],[],2025-10-30T03:53:18Z
1,19720061,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],[],[],[],2025-10-30T03:53:19Z
2,19818004,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],[],[],[],2025-10-30T03:53:20Z
3,19838068,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],[],[],[],2025-10-30T03:53:20Z
4,19880001,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],[],[],[],2025-10-30T03:53:21Z
5,19888010,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],[],[],[],2025-10-30T03:53:21Z
6,19890291,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],[],[],[],2025-10-30T03:53:22Z
7,19900204,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],[],[],[],2025-10-30T03:53:23Z
8,19900335,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],[],[],[],2025-10-30T03:53:23Z
9,19910182,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],[],[],[],2025-10-30T03:53:24Z


In [35]:
#@title 월간 집계
def aggregate_monthly_from_daily(daily_csv: str, out_csv: str):
    assert os.path.exists(daily_csv), f"daily csv 없음: {daily_csv}"
    df = pd.read_csv(daily_csv, dtype={"date": str})
    df["YYYYMM"] = df["date"].str.slice(0,7)
    for c in ["salesAmt","audiCnt","rank"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    grp = df.groupby(["YYYYMM","movieCd","movieNm"], as_index=False).agg(
        salesAmt_sum=("salesAmt","sum"),
        audiCnt_sum=("audiCnt","sum"),
        best_rank=("rank","min"),
    )
    grp = grp.sort_values(["YYYYMM","salesAmt_sum"], ascending=[True, False])
    grp.to_csv(out_csv, index=False, encoding="utf-8")

aggregate_monthly_from_daily(RAW_DAILY_CSV, MONTHLY_CSV)
print(f"월간 집계 완료 ✅ → {MONTHLY_CSV}")
display(pd.read_csv(MONTHLY_CSV).head(20))


월간 집계 완료 ✅ → /content/data/boxoffice_monthly.csv


,YYYYMM,movieCd,movieNm,salesAmt_sum,audiCnt_sum,best_rank
0,2015-01,20137048,국제시장,56449464034,7206312,1
1,2015-01,20143642,테이큰 3,15004684800,1850103,2
2,2015-01,20142407,오늘의 연애,13828469587,1752880,1
3,2015-01,20136888,강남 1970,12936882360,1602977,1
4,2015-01,20149951,빅 히어로,11655957733,1471198,1
5,2015-01,20149859,마다가스카의 펭귄,11217158559,1479639,3
6,2015-01,20143541,박물관이 살아있다 : 비밀의 무덤,7554160206,1019037,3
7,2015-01,20141111,"님아, 그 강을 건너지 마오",7217621800,913005,4
8,2015-01,20131262,허삼관,7132287969,919727,3
9,2015-01,20149265,기술자들,6055483535,762874,4


# 데이터 확인하기

In [28]:
df = pd.read_csv('/content/data/boxoffice_daily.csv', encoding = 'utf-8-sig')

In [32]:
df.tail()

,date,movieCd,movieNm,rank,rankInten,rankOldAndNew,salesAmt,salesShare,salesAcc,audiCnt,audiAcc,scrnCnt,showCnt,collected_at
29925,2023-03-12,20230200,똑똑똑,10,0,OLD,38634690,0.8,214876427,3722,21686,435,804,2025-10-30T03:42:51Z
29926,2023-03-12,20230209,앤트맨과 와스프: 퀀텀매니아,7,1,OLD,68999933,1.4,16259619630,6588,1542582,415,651,2025-10-30T03:42:51Z
29927,2023-03-12,20230495,아임 히어로 더 파이널,8,-1,OLD,153397000,3.2,4467381000,6394,185030,74,283,2025-10-30T03:42:51Z
29928,2023-03-12,20230533,"귀멸의 칼날: 상현집결, 그리고 도공 마을로",4,0,OLD,316000100,6.6,5492855100,27800,441839,322,1067,2025-10-30T03:42:51Z
29929,2023-03-12,20239573,서치 2,6,0,OLD,92436209,1.9,3892625634,8941,395500,424,659,2025-10-30T03:42:51Z


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29930 entries, 0 to 29929
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           29930 non-null  object 
 1   movieCd        29930 non-null  int64  
 2   movieNm        29930 non-null  object 
 3   rank           29930 non-null  int64  
 4   rankInten      29930 non-null  int64  
 5   rankOldAndNew  29930 non-null  object 
 6   salesAmt       29930 non-null  int64  
 7   salesShare     29930 non-null  float64
 8   salesAcc       29930 non-null  int64  
 9   audiCnt        29930 non-null  int64  
 10  audiAcc        29930 non-null  int64  
 11  scrnCnt        29930 non-null  int64  
 12  showCnt        29930 non-null  int64  
 13  collected_at   29930 non-null  object 
dtypes: float64(1), int64(9), object(4)
memory usage: 3.2+ MB


In [33]:
toprank_df = df.copy()
toprank_df.head()

,date,movieCd,movieNm,rank,rankInten,rankOldAndNew,salesAmt,salesShare,salesAcc,audiCnt,audiAcc,scrnCnt,showCnt,collected_at
0,2015-01-01,20130574,개를 훔치는 완벽한 방법,7,2,OLD,299346400,2.0,608002300,37741,80633,196,650,2025-10-30T02:54:13Z
1,2015-01-01,20137048,국제시장,1,0,OLD,6129982500,40.5,47219550096,751754,6097431,941,4650,2025-10-30T02:54:13Z
2,2015-01-01,20140226,호빗: 다섯 군대 전투,6,0,OLD,609678800,4.0,22038162144,69988,2579875,346,825,2025-10-30T02:54:13Z
3,2015-01-01,20141111,"님아, 그 강을 건너지 마오",4,1,OLD,1479015600,9.8,31324845079,178941,4025319,473,1979,2025-10-30T02:54:13Z
4,2015-01-01,20143344,눈의 여왕 2: 트롤의 마법거울,8,0,OLD,233190700,1.5,3856612400,30978,529298,246,415,2025-10-30T02:54:13Z


In [34]:
toprank_df['rank']

,rank
0,7
1,1
2,6
3,4
4,8
...,...
29925,10
29926,7
29927,8
29928,4


In [45]:
PATH = '/content/data/boxoffice_monthly.csv'

cols = ["YYYYMM","movieCd","movieNm","best_rank","audiCnt_sum","salesAmt_sum"]
toprank_df = pd.read_csv(PATH, dtype={"YYYYMM": str, "movieCd": str, "movieNm": str})
toprank_df = toprank_df.loc[:, cols]

toprank_df.shape
toprank_df.head(20)

,YYYYMM,movieCd,movieNm,best_rank,audiCnt_sum,salesAmt_sum
0,2015-01,20137048,국제시장,1,7206312,56449464034
1,2015-01,20143642,테이큰 3,2,1850103,15004684800
2,2015-01,20142407,오늘의 연애,1,1752880,13828469587
3,2015-01,20136888,강남 1970,1,1602977,12936882360
4,2015-01,20149951,빅 히어로,1,1471198,11655957733
5,2015-01,20149859,마다가스카의 펭귄,3,1479639,11217158559
6,2015-01,20143541,박물관이 살아있다 : 비밀의 무덤,3,1019037,7554160206
7,2015-01,20141111,"님아, 그 강을 건너지 마오",4,913005,7217621800
8,2015-01,20131262,허삼관,3,919727,7132287969
9,2015-01,20149265,기술자들,4,762874,6055483535


In [53]:
toprank_df["YYYYMM"] = (
    pd.to_datetime(toprank_df['YYYYMM'], errors='coerce')
    .dt.to_period("M")
    .astype(str)
    )

toprank_df["best_rank"] = pd.to_numeric(toprank_df["best_rank"], errors="coerce").astype("Int64")
toprank_df["audiCnt_sum"] = pd.to_numeric(toprank_df["audiCnt_sum"], errors="coerce")
toprank_df["salesAmt_sum"] = pd.to_numeric(toprank_df["salesAmt_sum"], errors="coerce")

toprank_df.dtypes
toprank_df.head(5)

,YYYYMM,movieCd,movieNm,best_rank,audiCnt_sum,salesAmt_sum
0,2015-01,20137048,국제시장,1,7206312,56449464034
1,2015-01,20143642,테이큰 3,2,1850103,15004684800
2,2015-01,20142407,오늘의 연애,1,1752880,13828469587
3,2015-01,20136888,강남 1970,1,1602977,12936882360
4,2015-01,20149951,빅 히어로,1,1471198,11655957733


In [54]:
# 3) 순위 밴딩 + 플래그 (판다스만)

# 순위 구간 레이블
toprank_df["rank_band"] = (
    pd.cut(
        toprank_df["best_rank"],
        bins=[-float("inf"), 1, 3, 5, 10, float("inf")],
        labels=["rank1", "rank2_3", "rank4_5", "rank6_10", "others"],
        right=True
    )
    .astype("object")
    .fillna("others")   # 결측/비정상 값은 others
)

# 빠른 플래그 4종
toprank_df = toprank_df.assign(
    is_top1 = toprank_df["best_rank"].eq(1),
    is_top3 = toprank_df["best_rank"].le(3),
    is_top5 = toprank_df["best_rank"].le(5),
    is_top10= toprank_df["best_rank"].le(10),
)

# NA → False 보정 (Int64 비교 시 NA가 나올 수 있음)
for c in ["is_top1","is_top3","is_top5","is_top10"]:
    toprank_df[c] = toprank_df[c].fillna(False)

# 확인(분포)
toprank_df["rank_band"].value_counts(dropna=False)


,count
rank_band,
rank6_10,1872
rank4_5,624
rank2_3,572
rank1,424


In [55]:
# 4) 기간 라벨: year/month/period (판다스만)

# YYYYMM → PeriodIndex로 연/월 뽑기
p = pd.PeriodIndex(toprank_df["YYYYMM"], freq="M")
toprank_df["year"]  = p.year.astype("Int64")
toprank_df["month"] = p.month.astype("Int64")

# period 라벨링
toprank_df["period"] = "other"
toprank_df.loc[toprank_df["year"].between(2020, 2022), "period"] = "pandemic"
toprank_df.loc[toprank_df["year"].between(2023, 2025), "period"] = "recovery"

# 확인
toprank_df["period"].value_counts()
toprank_df.head(5)


,YYYYMM,movieCd,movieNm,best_rank,audiCnt_sum,salesAmt_sum,rank_band,is_top1,is_top3,is_top5,is_top10,year,month,period
0,2015-01,20137048,국제시장,1,7206312,56449464034,rank1,True,True,True,True,2015,1,other
1,2015-01,20143642,테이큰 3,2,1850103,15004684800,rank2_3,False,True,True,True,2015,1,other
2,2015-01,20142407,오늘의 연애,1,1752880,13828469587,rank1,True,True,True,True,2015,1,other
3,2015-01,20136888,강남 1970,1,1602977,12936882360,rank1,True,True,True,True,2015,1,other
4,2015-01,20149951,빅 히어로,1,1471198,11655957733,rank1,True,True,True,True,2015,1,other


In [58]:
meta = pd.read_csv("/content/data/movie_meta_kobis.csv", dtype={"movieCd": str, "openDt": str})

meta["open_date"]   = pd.to_datetime(meta["openDt"], format="%Y%m%d", errors="coerce")
meta["open_yyyymm"] = meta["open_date"].dt.to_period("M").astype(str)
meta = meta.drop(columns=["openDt"])

toprank_df = toprank_df.merge(meta[["movieCd","open_date","open_yyyymm"]], on="movieCd", how="left")


In [61]:
# 6) 영화 단위 요약: 1위 개월수 / 피크 / 연속 스트릭 (판다스만, 수정판)

# 준비: YYYYMM → Period 변환, 1위만 추리기
p   = pd.PeriodIndex(toprank_df["YYYYMM"], freq="M")
tmp = toprank_df.assign(YYYYMM_p = p)

# (a) 영화별 1위 개월 수 + 피크(처음 1위 달)
g = (
    tmp.loc[tmp["is_top1"]]
       .groupby("movieCd", as_index=False)
       .agg(
           top1_months=("YYYYMM_p", "nunique"),
           peak_month =("YYYYMM_p", "min")
       )
)

# (b) 연속 1위 최장 스트릭 (월을 정수 키로 바꿔 연속성 검사)
def longest_streak(periods):
    if len(periods) == 0:
        return 0
    ps   = pd.PeriodIndex(sorted(periods), freq="M")
    key  = pd.Series(ps.year * 12 + ps.month, dtype="int64")   # 연*12+월 → 연속이면 차이가 1
    grp  = (key.diff().fillna(1) != 1).cumsum()                # 연속 끊기는 지점마다 그룹 증가
    return int(grp.value_counts().max())

streak_df = (
    tmp.loc[tmp["is_top1"]]
       .groupby("movieCd")["YYYYMM_p"]
       .apply(lambda s: longest_streak(s.tolist()))
       .reset_index(name="top1_streak_months")
)

# (c) (선택) 5번에서 만든 메타 일부 붙이기
meta_cols = ["movieCd","open_date","open_yyyymm"]
meta_use  = meta[meta_cols].drop_duplicates() if set(meta_cols).issubset(meta.columns) else None

# (d) 요약 테이블 결합
movie_rank_summary = g.merge(streak_df, on="movieCd", how="outer")
if meta_use is not None:
    movie_rank_summary = movie_rank_summary.merge(meta_use, on="movieCd", how="left")

# (e) 타입 정리
movie_rank_summary["peak_month"]         = movie_rank_summary["peak_month"].astype("string")
movie_rank_summary["top1_months"]        = movie_rank_summary["top1_months"].fillna(0).astype("Int64")
movie_rank_summary["top1_streak_months"] = movie_rank_summary["top1_streak_months"].fillna(0).astype("Int64")

# 확인
movie_rank_summary.head(10)


,movieCd,top1_months,peak_month,top1_streak_months,open_date,open_yyyymm
0,20098169,1,2015-06,1,NaT,NaT
1,20134798,2,2015-06,2,NaT,NaT
2,20136068,2,2015-12,2,NaT,NaT
3,20136888,2,2015-01,2,NaT,NaT
4,20137048,2,2015-01,2,NaT,NaT
5,20140110,2,2015-02,2,NaT,NaT
6,20140194,1,2016-05,1,NaT,NaT
7,20141384,1,2015-07,1,NaT,NaT
8,20142402,1,2015-05,1,NaT,NaT
9,20142407,1,2015-01,1,NaT,NaT


In [63]:
# 7) 마스터 전처리본 저장 (견고 버전: 누락 컬럼 자동 보강)

import pandas as pd

# movie_rank_summary가 없다면 빈 DF로 생성
if "movie_rank_summary" not in globals():
    movie_rank_summary = pd.DataFrame(columns=["movieCd","top1_months","top1_streak_months","peak_month"])

# 병합
pre_df = (
    toprank_df
      .merge(movie_rank_summary, on="movieCd", how="left")
      .copy()
)

# 기대 컬럼 목록
master_cols = [
    "YYYYMM","movieCd","movieNm","best_rank","rank_band",
    "is_top1","is_top3","is_top5","is_top10",
    "year","month","period",
    "audiCnt_sum","salesAmt_sum",
    "open_date","open_yyyymm",
    "top1_months","top1_streak_months","peak_month"
]

# 누락 컬럼 보강 (NA 채우기)
for c in master_cols:
    if c not in pre_df.columns:
        pre_df[c] = pd.NA

# 타입 살짝 정리(있을 때만)
for c in ["best_rank","year","month","top1_months","top1_streak_months"]:
    if c in pre_df.columns:
        pre_df[c] = pd.to_numeric(pre_df[c], errors="coerce").astype("Int64")

# 생성시간
pre_df["generated_at"] = pd.Timestamp.utcnow().isoformat(timespec="seconds")+"Z"

# 저장
out_path = "/content/data/monthly_ranked_preprocessed.csv"
pre_df.loc[:, master_cols + ["generated_at"]].to_csv(out_path, index=False, encoding="utf-8")
print("저장 완료 →", out_path)


저장 완료 → /content/data/monthly_ranked_preprocessed.csv


In [64]:
# 8) 서브셋 저장 (누락 플래그 자동 대비)

for name, cond_col, outfile in [
    ("rank1",  "is_top1",  "/content/data/monthly_rank1.csv"),
    ("top3",   "is_top3",  "/content/data/monthly_top3.csv"),
    ("top5",   "is_top5",  "/content/data/monthly_top5.csv"),
    ("top10",  "is_top10", "/content/data/monthly_top10.csv"),
]:
    if cond_col in pre_df.columns:
        pre_df.loc[pre_df[cond_col] == True].to_csv(outfile, index=False, encoding="utf-8")
        print(f"{name} 저장 → {outfile}")
    else:
        print(f"{name} 건너뜀 (컬럼 없음: {cond_col})")

# 영화 단위 요약도 있으면 저장
if "movie_rank_summary" in globals() and len(movie_rank_summary):
    movie_rank_summary.to_csv("/content/data/movie_rank_summary.csv", index=False, encoding="utf-8")
    print("movie_rank_summary 저장")
else:
    print("movie_rank_summary 없음 → 건너뜀")


rank1 저장 → /content/data/monthly_rank1.csv
top3 저장 → /content/data/monthly_top3.csv
top5 저장 → /content/data/monthly_top5.csv
top10 저장 → /content/data/monthly_top10.csv
movie_rank_summary 저장


In [65]:
# 9) 빠른 검증 (존재하는 것만 체크)

print("rank_band 분포:")
if "rank_band" in pre_df.columns:
    print(pre_df["rank_band"].value_counts(dropna=False).head(10))
else:
    print("rank_band 없음")

print("\n포함관계 평균치(비율):")
for c in ["is_top1","is_top3","is_top5","is_top10"]:
    if c in pre_df.columns:
        print(c, ":", pre_df[c].fillna(False).mean())
    else:
        print(c, ": (없음)")

# 저장본 샘플
pd.read_csv("/content/data/monthly_ranked_preprocessed.csv").head(5)


rank_band 분포:
rank_band
rank6_10    1872
rank4_5      624
rank2_3      572
rank1        424
Name: count, dtype: int64

포함관계 평균치(비율):
is_top1 : 0.12142038946162657
is_top3 : 0.2852233676975945
is_top5 : 0.4639175257731959
is_top10 : 1.0


,YYYYMM,movieCd,movieNm,best_rank,rank_band,is_top1,is_top3,is_top5,is_top10,year,month,period,audiCnt_sum,salesAmt_sum,open_date,open_yyyymm,top1_months,top1_streak_months,peak_month,generated_at
0,2015-01,20137048,국제시장,1,rank1,True,True,True,True,2015,1,other,7206312,56449464034,NaN,NaN,2.0,2.0,2015-01,2025-10-30T05:01:52+00:00Z
1,2015-01,20143642,테이큰 3,2,rank2_3,False,True,True,True,2015,1,other,1850103,15004684800,NaN,NaN,NaN,NaN,NaN,2025-10-30T05:01:52+00:00Z
2,2015-01,20142407,오늘의 연애,1,rank1,True,True,True,True,2015,1,other,1752880,13828469587,NaN,NaN,1.0,1.0,2015-01,2025-10-30T05:01:52+00:00Z
3,2015-01,20136888,강남 1970,1,rank1,True,True,True,True,2015,1,other,1602977,12936882360,NaN,NaN,2.0,2.0,2015-01,2025-10-30T05:01:52+00:00Z
4,2015-01,20149951,빅 히어로,1,rank1,True,True,True,True,2015,1,other,1471198,11655957733,NaN,NaN,2.0,2.0,2015-01,2025-10-30T05:01:52+00:00Z


# 데이터 분석

In [67]:
toprank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3492 entries, 0 to 3491
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   YYYYMM        3492 non-null   object        
 1   movieCd       3492 non-null   object        
 2   movieNm       3492 non-null   object        
 3   best_rank     3492 non-null   Int64         
 4   audiCnt_sum   3492 non-null   int64         
 5   salesAmt_sum  3492 non-null   int64         
 6   rank_band     3492 non-null   object        
 7   is_top1       3492 non-null   boolean       
 8   is_top3       3492 non-null   boolean       
 9   is_top5       3492 non-null   boolean       
 10  is_top10      3492 non-null   boolean       
 11  year          3492 non-null   Int64         
 12  month         3492 non-null   Int64         
 13  period        3492 non-null   object        
 14  open_date     0 non-null      datetime64[ns]
 15  open_yyyymm   3492 non-null   object  

# 2015~2023 전체 기간별 분석

## 역대 TOP 10

### 역대 관객수 TOP 10

In [102]:
# 1) (필요시) 로드
import pandas as pd, plotly.express as px

daily = pd.read_csv("/content/data/boxoffice_daily.csv", dtype={"movieNm": str})
daily["date"] = pd.to_datetime(daily["date"], errors="coerce")
daily["audiCnt"] = pd.to_numeric(daily["audiCnt"], errors="coerce")


In [103]:
# 2) 제목 기준 합계 → TOP10
tot_name = (daily
            .dropna(subset=["movieNm"])
            .groupby("movieNm", as_index=False)["audiCnt"]
            .sum()
            .rename(columns={"audiCnt":"audi_total"}))

top10_name = (tot_name
              .sort_values("audi_total", ascending=False)
              .head(10)
              .sort_values("audi_total"))  # 가로막대 위→아래 증가용


In [104]:
# 3) 차트
period_label = f"{daily['date'].min().strftime('%Y-%m')} ~ {daily['date'].max().strftime('%Y-%m')}"

fig = px.bar(
    top10_name,
    x="audi_total", y="movieNm",
    orientation="h",
    text="audi_total",
    labels={"movieNm":"영화(제목 기준 통합)", "audi_total":"누적 관객수(일별 합)"},
    title="역대 관객수 TOP 10 (movieNm 기준 통합)"
)
fig.update_traces(textposition="outside", cliponaxis=False, texttemplate="%{text:,}")
fig.update_layout(height=520, margin=dict(l=320, r=40, t=70, b=120), bargap=0.2)

fig.add_annotation(x=0, y=-0.14, xref="paper", yref="paper",
                   xanchor="left", yanchor="top",
                   text=f"분석 기준 기간: {period_label}", showarrow=False)
fig.show()


### 역대 매출액 TOP 10

In [105]:
import pandas as pd
import plotly.express as px

# 1) 로드 & 전처리
daily = pd.read_csv("/content/data/boxoffice_daily.csv", dtype={"movieNm": str})
daily["date"] = pd.to_datetime(daily["date"], errors="coerce")
daily["salesAmt"] = pd.to_numeric(daily["salesAmt"], errors="coerce")  # 매출액

# 2) 제목(movieNm) 기준 합계 → TOP10
tot_sales = (daily
             .dropna(subset=["movieNm"])
             .groupby("movieNm", as_index=False)["salesAmt"]
             .sum()
             .rename(columns={"salesAmt":"sales_total"}))

top10_sales = (tot_sales
               .sort_values("sales_total", ascending=False)
               .head(10)
               .sort_values("sales_total"))  # 가로 막대 위→아래 증가용

# 3) 기간 라벨
period_label = f"{daily['date'].min().strftime('%Y-%m')} ~ {daily['date'].max().strftime('%Y-%m')}"

# 4) 차트
fig = px.bar(
    top10_sales,
    x="sales_total", y="movieNm",
    orientation="h",
    text="sales_total",
    labels={"movieNm":"영화(제목 기준 통합)", "sales_total":"누적 매출액(일별 합)"},
    title="역대 매출액 TOP 10 (movieNm 기준 통합)"
)
fig.update_traces(textposition="outside", cliponaxis=False, texttemplate="%{text:,}")  # 천단위 콤마
fig.update_layout(
    height=max(520, 38*len(top10_sales)+160),
    margin=dict(l=320, r=40, t=70, b=120),
    bargap=0.2
)
fig.add_annotation(
    x=0, y=-0.14, xref="paper", yref="paper",
    xanchor="left", yanchor="top",
    text=f"분석 기준 기간: {period_label}",
    showarrow=False
)
fig.show()


#### 관객 X 매출액 차이 구분 산점도

In [106]:
import pandas as pd
import plotly.express as px
import numpy as np

# 1) 로드 & 전처리
daily = pd.read_csv("/content/data/boxoffice_daily.csv", dtype={"movieNm": str})
daily["date"] = pd.to_datetime(daily["date"], errors="coerce")
daily["audiCnt"] = pd.to_numeric(daily["audiCnt"], errors="coerce")
daily["salesAmt"] = pd.to_numeric(daily["salesAmt"], errors="coerce")

# 2) movieNm 기준 합계 → 평균 티켓가
agg = (daily.dropna(subset=["movieNm"])
            .groupby("movieNm", as_index=False)
            .agg(audi_total=("audiCnt","sum"),
                 sales_total=("salesAmt","sum"),
                 days_in_top10=("movieNm","size")))

# 평균 티켓가(원) = 매출합 / 관객합 (0/NaN 방지)
agg = agg[(agg["audi_total"] > 0) & (agg["sales_total"] > 0)].copy()
agg["avg_price"] = agg["sales_total"] / agg["audi_total"]

# (선택) 너무 작은 작품 제외해 노이즈 완화 (원하면 주석 해제)
# agg = agg[agg["audi_total"] >= 50_000]  # 예: Top10에 잡힌 관객수 5만 이상

# 3) 전체 평균선(참고용)
global_avg = (agg["sales_total"].sum() / agg["audi_total"].sum())
xmin, xmax = agg["audi_total"].min(), agg["audi_total"].max()

# 4) 산점도: x=관객합, y=매출합, 색=평균 티켓가(원), 점크기=Top10 등장일수
period_label = f"{daily['date'].min():%Y-%m} ~ {daily['date'].max():%Y-%m}"

fig = px.scatter(
    agg,
    x="audi_total", y="sales_total",
    color="avg_price", size="days_in_top10",
    hover_name="movieNm",
    hover_data={
        "audi_total": ":,", "sales_total": ":,",
        "avg_price": ":,.0f", "days_in_top10": True
    },
    labels={
        "audi_total":"누적 관객수(Top10 합)",
        "sales_total":"누적 매출액(Top10 합)",
        "avg_price":"평균 티켓가(원)",
        "days_in_top10":"Top10 등장일수"
    },
    title="관객수 vs 매출액 산점도 (색=평균 티켓가, 크기=Top10 등장일수)"
)

# 보기 좋게 다듬기
fig.update_layout(
    margin=dict(l=80, r=40, t=70, b=90),
    coloraxis_colorbar=dict(title="평균 티켓가(원)"),
)
# 전체 평균 티켓가 기준의 기준선 y = (global_avg) * x
fig.add_shape(
    type="line", x0=xmin, y0=global_avg*xmin, x1=xmax, y1=global_avg*xmax,
    xref="x", yref="y"
)
fig.add_annotation(
    x=xmax, y=global_avg*xmax, xref="x", yref="y",
    text=f"전체 평균 ~ {global_avg:,.0f}원", showarrow=False, xanchor="right", yanchor="bottom"
)

# (선택) 로그축을 원하면 주석 해제
# fig.update_xaxes(type="log")
# fig.update_yaxes(type="log")

# 하단 기간 표기
fig.add_annotation(
    x=0, y=-0.18, xref="paper", yref="paper",
    xanchor="left", yanchor="top",
    text=f"분석 기준 기간: {period_label}",
    showarrow=False
)

fig.show()


### 연도별 최다 흥행 영화

#### 관객수 기준

In [111]:
import pandas as pd
import plotly.express as px

# 1) 로드 & 전처리
daily = pd.read_csv("/content/data/boxoffice_daily.csv", dtype={"movieNm": str})
daily["date"] = pd.to_datetime(daily["date"], errors="coerce")
daily["audiCnt"] = pd.to_numeric(daily["audiCnt"], errors="coerce")

# 분석 기간 라벨
period_label = f"{daily['date'].min():%Y-%m} ~ {daily['date'].max():%Y-%m}"

# 2) 연도 컬럼
daily["year"] = daily["date"].dt.year

# 3) 연-영화별 관객 합 (movieNm 기준 통합)
year_movie_audi = (daily
                   .dropna(subset=["movieNm"])
                   .groupby(["year","movieNm"], as_index=False)["audiCnt"].sum()
                   .rename(columns={"audiCnt":"audi_total"}))

# 4) 각 연도별 관객 1위 영화 추출
top_by_year_audi = (year_movie_audi
                    .sort_values(["year","audi_total"], ascending=[True, False])
                    .drop_duplicates(subset=["year"], keep="first")
                    .sort_values("year"))

# 5) 차트 (막대: y=관객수, x=연도, 텍스트=영화명)
fig = px.bar(
    top_by_year_audi,
    x="year", y="audi_total",
    text="movieNm",
    hover_data={"movieNm": True, "audi_total":":,"},
    labels={"year":"연도", "audi_total":"해당 연도 관객 합(Top10 범위)", "movieNm":"최다 관객 영화"},
    title="연도별 최다 흥행 영화 (관객수)"
)

# 보기 좋게
fig.update_traces(textposition="outside", cliponaxis=False)
fig.update_layout(
    xaxis=dict(dtick=1),
    margin=dict(l=80, r=40, t=70, b=110),
    bargap=0.2
)

# 하단 기간 표기
fig.add_annotation(
    x=0, y=-0.18, xref="paper", yref="paper",
    xanchor="left", yanchor="top",
    text=f"분석 기준 기간: {period_label}  ·  지표: 일별 Top10 관객 합계",
    showarrow=False
)

fig.show()


#### 매출액 기준

In [109]:
import pandas as pd
import plotly.express as px

# 1) 로드 & 전처리
daily = pd.read_csv("/content/data/boxoffice_daily.csv", dtype={"movieNm": str})
daily["date"] = pd.to_datetime(daily["date"], errors="coerce")
daily["salesAmt"] = pd.to_numeric(daily["salesAmt"], errors="coerce")

# 분석 기간 라벨
period_label = f"{daily['date'].min():%Y-%m} ~ {daily['date'].max():%Y-%m}"

# 2) 연도 컬럼
daily["year"] = daily["date"].dt.year

# 3) 연-영화별 매출 합 (movieNm 기준 통합)
year_movie = (daily
              .dropna(subset=["movieNm"])
              .groupby(["year","movieNm"], as_index=False)["salesAmt"].sum()
              .rename(columns={"salesAmt":"sales_total"}))

# 4) 각 연도별 매출 1위 영화 추출
#    방법: 연도 내 매출 내림차순 → 연도별 첫 행만 선택
top_by_year = (year_movie
               .sort_values(["year","sales_total"], ascending=[True, False])
               .drop_duplicates(subset=["year"], keep="first")
               .sort_values("year"))

# 5) 차트 (막대: y=매출, x=연도, 텍스트에 영화명)
fig = px.bar(
    top_by_year,
    x="year", y="sales_total",
    text="movieNm",
    hover_data={"movieNm": True, "sales_total":":,"},
    labels={"year":"연도", "sales_total":"해당 연도 매출 합(Top10 범위)", "movieNm":"최다 흥행 영화"},
    title="연도별 최다 흥행 영화 (매출액)"
)

# 보기 좋게
fig.update_traces(textposition="outside", cliponaxis=False)
fig.update_layout(
    xaxis=dict(dtick=1),
    margin=dict(l=80, r=40, t=70, b=110),
    bargap=0.2
)

# 하단 기간 표기
fig.add_annotation(
    x=0, y=-0.18, xref="paper", yref="paper",
    xanchor="left", yanchor="top",
    text=f"분석 기준 기간: {period_label}  ·  지표: 일별 Top10 매출 합계",
    showarrow=False
)

fig.show()


### 최장기간 1위 유지 영화 Top20

In [101]:
import pandas as pd
import plotly.express as px

# 1) 로드 & 전처리
daily = pd.read_csv("/content/data/boxoffice_daily.csv", dtype={"movieCd": str, "movieNm": str})
daily["date"] = pd.to_datetime(daily["date"], errors="coerce")
daily["rank"] = pd.to_numeric(daily["rank"], errors="coerce")

# 2) 일일 1위만 추출 → 영화별 연속일수(streak) 계산
top1 = (daily.loc[daily["rank"].eq(1), ["movieCd","movieNm","date"]]
              .sort_values(["movieCd","date"])
              .dropna(subset=["date"]))

# 연속성 판별: 이전 날짜와 차이가 1일이 아니면 새 스트릭
gap = top1.groupby("movieCd")["date"].diff().dt.days.ne(1)  # 첫 행은 True
top1["streak_id"] = gap.groupby(top1["movieCd"]).cumsum()   # 영화별 누적 합 → 스트릭 번호

# 3) 스트릭 요약(영화×streak_id): 길이/시작/끝/영화명
streaks = (top1.groupby(["movieCd","streak_id"], as_index=False)
                .agg(
                    days_count=("date","size"),
                    start_date=("date","min"),
                    end_date=("date","max"),
                    movieNm=("movieNm", "first")
                ))

# 4) Top20 선별 (동률 시 더 긴 기간이 먼저, 그다음 시작일이 이른 순)
top20 = (streaks.sort_values(["days_count","start_date"], ascending=[False, True])
                 .head(20)
                 .assign(
                     label=lambda d: d["movieNm"].fillna(d["movieCd"]) + " · " \
                                     + d["start_date"].dt.strftime("%Y-%m-%d") + " ~ " \
                                     + d["end_date"].dt.strftime("%Y-%m-%d")
                 ))

# y축 순서(짧→길 정렬로 위에서 아래로 보이게)
y_order = top20.sort_values("days_count")["label"].tolist()

# 전체 기간 라벨(일별 데이터 기준)
period_label = f"{daily['date'].min().strftime('%Y-%m')} ~ {daily['date'].max().strftime('%Y-%m')}"

# 5) 차트
fig = px.bar(
    top20,
    x="days_count", y="label",
    orientation="h",
    text="days_count",
    labels={"days_count":"연속 1위 일수", "label":"영화 · 기간"},
    title="일별 최장기간 1위 유지 영화 Top 20"
)
fig.update_yaxes(categoryorder="array", categoryarray=y_order)
fig.update_traces(textposition="outside", cliponaxis=False, texttemplate="%{x}일")

# 보기 좋게 레이아웃
fig.update_layout(
    height=max(520, 34*len(top20)+180),
    margin=dict(l=340, r=40, t=70, b=120),
    bargap=0.2
)
fig.update_xaxes(range=[0, int(top20["days_count"].max()) + 1])

# 하단 기간 표기
fig.add_annotation(
    x=0, y=-0.15, xref="paper", yref="paper",
    xanchor="left", yanchor="top",
    text=f"분석 기준 기간: {period_label}",
    showarrow=False
)

fig.show()


### 연도별 Top10 빈도

#### 관람객수

In [118]:
# freq_topk가 이미 만들어졌다고 가정(없으면 아래 주석 해제해서 만들기)
# K = 10
# top10 = daily[daily["rank"].le(10)].dropna(subset=["movieNm","date"])
# freq = (top10.groupby(["year","movieNm"], as_index=False)
#               .agg(days_in_top10=("date","nunique")))
# freq_topk = (freq.sort_values(["year","days_in_top10"], ascending=[True, False])
#                  .groupby("year").head(K))

import plotly.express as px

# x축(영화) 정렬: 전체 기간 합이 큰 영화 순서로 왼→오
movie_order = (freq_topk.groupby("movieNm")["days_in_top10"].sum()
               .sort_values(ascending=False).index.tolist())

# (없을 수도 있는) K 기본값 처리
if "K" not in globals():
    K = 10

fig = px.density_heatmap(
    freq_topk,
    x="movieNm",            # ← 가로축에 영화
    y="year",               # ← 세로축에 연도
    z="days_in_top10",
    color_continuous_scale="Blues",
    labels={"movieNm":"영화", "year":"연도", "days_in_top10":"Top10 등장 일수"},
    title=f"연도별 Top10 빈도(일수) 히트맵 — 각 연도 상위 {K}편 (가로 배열)"
)

# ── 수정 포인트 ─────────────────────────────────────────────
# 1) y축: 최신→과거 순으로 지정 (맨 아래가 2014)
y_order = sorted(freq_topk["year"].unique(), reverse=True)
fig.update_yaxes(categoryorder="array", categoryarray=y_order, dtick=1)  # autorange 제거
# ────────────────────────────────────────────────────────────

# x축/레이아웃 다듬기
fig.update_xaxes(categoryorder="array", categoryarray=movie_order, tickangle=-40)
fig.update_layout(
    margin=dict(l=80, r=40, t=70, b=120),
    height=600,
    width=max(1000, 22*len(movie_order) + 200)  # 영화 많으면 가로폭 자동 확대
)

fig.show()


### 연간 관람객수

# 월별분석 (숨김)

## 월별 1위 영화 분석

### 1~12월 분포 히스토그램

In [71]:
!pip install plotly

In [74]:
# month_label 만들었다고 가정
# month_order = [f"{i:02d}월" for i in range(1,13)]

fig = px.histogram(
    top1, y="month_label",
    category_orders={"month_label": month_order},
    text_auto=True
)
fig.update_layout(xaxis_title="Count of #1 movies", yaxis_title="Month")
fig.show()


In [73]:
import plotly.express as px

cnt = (top1["YYYYMM"]
       .value_counts()
       .rename_axis("YYYYMM")
       .reset_index(name="count")
       .sort_values("YYYYMM"))

fig = px.bar(
    cnt, x="YYYYMM", y="count", text="count"
)
fig.update_layout(
    xaxis_title="YYYY-MM",
    yaxis_title="Count of #1 movies",
    xaxis_tickangle=-45
)
fig.show()


### 월간 최장기간 1위 영화, 최다 관람객 수 영화

In [82]:
import pandas as pd
import plotly.express as px

# ── 요약 테이블이 없으면 만들기 ───────────────────────────────
if "movie_rank_summary" not in globals():
    p   = pd.PeriodIndex(toprank_df["YYYYMM"], freq="M")
    tmp = toprank_df.assign(YYYYMM_p=p).loc[toprank_df["is_top1"]].copy()

    g = (tmp.groupby("movieCd", as_index=False)
            .agg(top1_months=("YYYYMM_p","nunique"),
                 peak_month =("YYYYMM_p","min")))

    def longest_streak(periods):
        if len(periods)==0: return 0
        ps  = pd.PeriodIndex(sorted(periods), freq="M")
        key = pd.Series(ps.year*12 + ps.month, dtype="int64")
        grp = (key.diff().fillna(1) != 1).cumsum()
        return int(grp.value_counts().max())

    streak = (tmp.groupby("movieCd")["YYYYMM_p"]
                .apply(lambda s: longest_streak(s.tolist()))
                .reset_index(name="top1_streak_months"))

    movie_rank_summary = g.merge(streak, on="movieCd", how="outer")
    movie_rank_summary["peak_month"] = movie_rank_summary["peak_month"].astype("string")
    movie_rank_summary[["top1_months","top1_streak_months"]] = (
        movie_rank_summary[["top1_months","top1_streak_months"]].fillna(0).astype("Int64")
    )

# ── 영화명 붙이고 Top20 추출 ────────────────────────────────
names = (toprank_df.groupby("movieCd", as_index=False)["movieNm"]
                   .agg(lambda s: s.dropna().iloc[0] if len(s.dropna()) else None))

top20 = (movie_rank_summary.merge(names, on="movieCd", how="left")
         .sort_values(["top1_streak_months","top1_months","peak_month"],
                      ascending=[False, False, True])
         .head(20)
         .assign(movie_label=lambda df: df["movieNm"].fillna(df["movieCd"])))

y_order = top20.sort_values("top1_streak_months")["movie_label"].tolist()

# ── 전체 기간 라벨 만들기 (예: 2015-01 ~ 2023-03) ──────────────
p_all = pd.PeriodIndex(toprank_df["YYYYMM"], freq="M")
period_label = f"{p_all.min().strftime('%Y-%m')} ~ {p_all.max().strftime('%Y-%m')}"

# ── 차트 ────────────────────────────────────────────────────

# 텍스트를 바깥에 표시 + 축을 넘어가도 보이게
fig.update_traces(textposition="outside", cliponaxis=False, texttemplate="%{x}개월")

# 항목 수에 따라 높이 자동 조절 (막대당 34px 정도)
n = len(top20)
fig.update_layout(
    height= max(400, 34*n + 180),
    margin=dict(l=260, r=40, t=60, b=120),
    uniformtext_minsize=10, uniformtext_mode="hide"
)

# x축에 여유 공간 줘서 텍스트가 오른쪽에 안 겹치게
xmax = float(top20["top1_streak_months"].max() or 0)
fig.update_xaxes(range=[0, xmax + 0.8])

# 하단 기간 라벨 위치 조금 더 아래로
fig.update_layout(annotations=[
    dict(x=0, y=-0.14, xref="paper", yref="paper",
         xanchor="left", yanchor="top",
         text=f"전체 기간 {period_label}", showarrow=False)
])

fig.show()


### 개봉 이후 10일 내로 1위한 영화

In [92]:
import pandas as pd

# 로드
daily = pd.read_csv("/content/data/boxoffice_daily.csv", dtype={"movieCd": str, "movieNm": str})
meta  = pd.read_csv("/content/data/movie_meta_kobis.csv", dtype={"movieCd": str, "openDt": str})

# 타입
daily["date"] = pd.to_datetime(daily["date"], errors="coerce")
daily["rank"] = pd.to_numeric(daily["rank"], errors="coerce")
meta["open_date"] = pd.to_datetime(meta["openDt"], format="%Y%m%d", errors="coerce")

# 최초 1위 날짜
no1 = daily.loc[daily["rank"].eq(1), ["movieCd","movieNm","date"]].copy()
first_no1 = (no1.sort_values(["movieCd","date"])
             .groupby("movieCd", as_index=False)
             .agg(first_no1_date=("date","min")))

# 개봉일 결측 보완: 일별 최초 등장일로 대체
first_seen = (daily.sort_values(["movieCd","date"])
                 .groupby("movieCd", as_index=False)
                 .agg(first_seen_date=("date","min")))

base = (first_no1
        .merge(meta[["movieCd","open_date"]], on="movieCd", how="left")
        .merge(first_seen, on="movieCd", how="left"))

base["open_date_filled"] = base["open_date"].fillna(base["first_seen_date"])

# 최초 1위 시점의 영화명 붙이기
names = (no1.merge(first_no1, left_on=["movieCd","date"], right_on=["movieCd","first_no1_date"], how="right")
            .rename(columns={"movieNm":"movieNm_at_first1"}))[["movieCd","movieNm_at_first1"]]
base = base.merge(names, on="movieCd", how="left")

# 개봉→첫 1위 소요일 계산
base["days_to_first1"] = (base["first_no1_date"] - base["open_date_filled"]).dt.days
base = base[base["days_to_first1"].ge(0) & base["days_to_first1"].notna()].copy()

# 1~10일 버킷
df10 = base[base["days_to_first1"].between(1,10)].copy()
df10["bucket"] = df10["days_to_first1"].astype(int)
df10["movie_label"] = df10["movieNm_at_first1"].fillna(df10["movieCd"])

period_label = f"{daily['date'].min().strftime('%Y-%m')} ~ {daily['date'].max().strftime('%Y-%m')}"


In [100]:
import plotly.express as px
import pandas as pd

# 1) 개봉 후 1일 만에 1위 달성한 영화만 추출
day1 = base.loc[base["days_to_first1"].eq(1)].copy()

# 2) 연도 컬럼(첫 1위 달성 연도) 추가
day1["year"] = day1["first_no1_date"].dt.year

# 3) 연도별 개수 집계
cnt_year = (day1.groupby("year").size()
              .rename("count").reset_index()
              .sort_values("year"))

# 4) 막대 차트
fig = px.bar(
    cnt_year, x="year", y="count", text="count",
    labels={"year":"연도(첫 1위 달성 연도)", "count":"개봉 후 1일 내 1위 달성 영화 수"},
    title="연도별 / 개봉 후 1일 내 1위 달성 영화 수 분포"
)
fig.update_traces(textposition="outside", cliponaxis=False)
fig.update_layout(xaxis=dict(dtick=1))

# (선택) 하단 기간 표기: 일별 전체 기간 대신, 1일 달성 표본의 기간으로 표기하려면 아래 사용
period_label_day1 = (
    f"{day1['first_no1_date'].min().strftime('%Y-%m')} ~ "
    f"{day1['first_no1_date'].max().strftime('%Y-%m')}"
) if not day1.empty else period_label

fig.add_annotation(
    x=0, y=-0.18, xref="paper", yref="paper",
    xanchor="left", yanchor="top",
    text=f"분석 기준 기간: {period_label_day1}",
    showarrow=False
)
fig.show()


In [99]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 버킷별 상위 N개 (기본 15)
N = 15
bucket_range = range(1, 11)

# 각 버킷 데이터 정렬: 달성일 빠른 순 → 영화명
lists = (
    df10.sort_values(["bucket", "first_no1_date", "movie_label"])
        .groupby("bucket")
        .apply(lambda g: g.head(N))
        .reset_index(drop=True)
)

# 1~10일, 세로로 10개 서브플롯(각각 Table)
fig = make_subplots(
    rows=10, cols=1,
    specs=[[{"type":"table"}]]*10,
    vertical_spacing=0.03,
    subplot_titles=[f"{b}일" for b in bucket_range]
)

for i, b in enumerate(bucket_range, start=1):
    sub = lists[lists["bucket"].eq(b)].copy()
    if sub.empty:
        titles = ["(없음)"]
        dates  = [""]
    else:
        titles = sub["movie_label"].tolist()
        dates  = sub["first_no1_date"].dt.strftime("%Y-%m-%d").tolist()

    fig.add_trace(
        go.Table(
            header=dict(
                values=["영화제목", "1위 달성일"],
                fill_color="#E6EEF8",
                align=["left","center"],
                font=dict(size=12, color="#2b2b2b"),
                height=28
            ),
            cells=dict(
                values=[titles, dates],
                align=["left","center"],
                font=dict(size=11),
                height=24,
                fill_color=[["#F9FBFD"]*len(titles), ["#FFFFFF"]*len(dates)],
            ),
            columnwidth=[0.72, 0.28],   # 제목/날짜 폭 비율
        ),
        row=i, col=1
    )

# 레이아웃 다듬기
fig.update_layout(
    title="개봉 후 1~10일 내 1위 달성 영화 목록 (버킷별 상위 15편)",
    height= max(1400, 10 * (N*24 + 70)),   # 표 높이에 따라 자동 늘림
    margin=dict(l=60, r=40, t=70, b=90),
)

# 하단 기간 표기
fig.add_annotation(
    x=0, y=-0.06, xref="paper", yref="paper",
    xanchor="left", yanchor="top",
    text=f"분석 기준 기간: {period_label}",
    showarrow=False
)

fig.show()
